# Stock Market Indicators

### Initialization

In [ ]:
import os 
import pandas as pd
import numpy as np 
import datetime as dt
import matplotlib.pyplot as plt
import getData

### Setup

In [ ]:
path1      = r"D:\Desky\data"
start_date = "2008-01-01"
end_date   = "2009-12-31"
symbols    = ["GOOG"]

In [ ]:
dates = pd.date_range(start_date, end_date)

stockHigh = getData(symbols, dates, addSPY=False, colname="High")
stockHigh.sort_index(); 
stockHigh = stockHigh.fillna(method='ffill')
stockHigh = stockHigh.fillna(method='bfill')

stockLow = getData(symbols, dates, addSPY=False, colname="Low")
stockLow.sort_index(); 
stockLow = stockLow.fillna(method='ffill')
stockLow = stockLow.fillna(method='bfill')

stock = getData(symbols, dates, addSPY=False) #Adjusted Close
stock = stock.sort_index()
stock = stock.fillna(method='ffill')
stock = stock.fillna(method='bfill')

### Bollinger Bands

In [ ]:
def bollinger_bands(stock, window=20, kconst=2):
    stock_sma  = stock.rolling(window).mean()
    stock_sgm  = stock.rolling(window).std()
    stock_bbUp = stock_sma + kconst*stock_sgm
    stock_bbDw = stock_sma - kconst*stock_sgm
    stock_bPb  = (stock-stock_bbDw)/(stock_bbUp - stock_bbDw)
    
    plt.figure(1)
    plt.subplot(3,1,1)
    plt.plot(stock,'k-')
    plt.title("Stock Prices")
    plt.grid(True)
    plt.legend(["stock"])
    plt.xlabel("Time")
    plt.ylabel("Prices")
    
    plt.subplot(3,1,2)
    plt.plot(stock,'k-')
    plt.plot(stock_bbUp,'r-')
    plt.plot(stock_bbDw,'b-')
    plt.title("Bollinger Bands")
    plt.grid(True)
    plt.legend(["stock","stock_sma+2o","stock_sma-2o"])
    plt.xlabel("Time")
    plt.ylabel("Prices")
    
    plt.subplot(3,1,3)
    index = np.arange(len(stock_bPb))
    plt.bar(index,stock_bPb["stock"])
    plt.axhline(y=1, linewidth=1, color = 'r', linestyle='--')
    plt.axhline(y=0, linewidth=1, color = 'b', linestyle='--')    
    plt.title("Bollinger Bands Percent")
    plt.grid(True)
    #plt.legend(["stock"])
    plt.xlabel("Time")
    plt.ylabel("b-Percent (b%)")
    
    plt.savefig('Figure1.png', dpi=600, bbox_inches='tight')
    return None


### Moving Average Convegence Divergence

In [ ]:
def macd_indicator(stock, fast_alpha=12, slow_alpha=26, sgnl_alpa=9):
    fast12ema = stock.ewm(span=fast_alpha, min_periods=0, adjust=False, ignore_na=False).mean()
    slow26ema = stock.ewm(span=slow_alpha, min_periods=0, adjust=False, ignore_na=False).mean()
    macdline = fast12ema - slow26ema #blue line
    signal   = macdline.ewm(span=sgnl_alpa, min_periods=0, adjust=False, ignore_na=False).mean() #red line
    macdbar  = macdline - signal #bar graph

    plt.figure(2)
    plt.subplot(3,1,1)
    plt.plot(stock,'k-')
    plt.plot(fast12ema,'r-')
    plt.plot(slow26ema,'b-')
    plt.title("Stock Prices")
    plt.grid(True)
    plt.legend(["stock","EWMA[12]","EWMA[26]"])
    plt.xlabel("Time")
    plt.ylabel("Prices")
    
    plt.subplot(3,1,2)
    plt.plot(signal,'r-')
    plt.plot(macdline,'b-')
    plt.title("MACD Charts")
    plt.grid(True)
    plt.legend(["Signal", "MACD line"])
    plt.xlabel("Time")
    plt.ylabel("Prices")
    
    plt.subplot(3,1,3)
    index = np.arange(len(macdbar))
    plt.bar(index, macdbar["stock"].values)
    plt.title("MACD Bar")
    plt.grid(True)
    plt.legend(["MACD Bar"])
    plt.xlabel("Time")
    plt.ylabel("MACD:Signal")
    plt.savefig('Figure2.png', dpi=600, bbox_inches='tight')
    return None    

### Relative Strength

In [ ]:
def rsi_indicator(stock, rsi_days=14):
    RSI_Days = rsi_days
    rsi_change = stock.values[1:] - stock.values[:-1]
    up_mve = rsi_change.copy()
    dn_mve = rsi_change.copy()
    up_mve[up_mve < 0] = 0
    dn_mve[dn_mve > 0] = 0
    dn_mve = np.abs(dn_mve)
    avgup_mve = np.zeros((len(rsi_change),1))
    avgdn_mve = np.zeros((len(rsi_change),1))
    avgup_mve[0] = np.mean(up_mve[0:RSI_Days])
    avgdn_mve[0] = np.mean(dn_mve[0:RSI_Days])
    for i in range(1,len(rsi_change)):
        avgup_mve[i] = ((RSI_Days-1)*avgup_mve[i-1] + up_mve[i])/RSI_Days
        avgdn_mve[i] = ((RSI_Days-1)*avgdn_mve[i-1] + dn_mve[i])/RSI_Days
    RS  = avgup_mve/avgdn_mve
    RSI = 100 - (100/(RS + 1))
    
    plt.figure(3)
    plt.subplot(2,1,1)
    plt.plot(stock,'k-')
    plt.title("Stock Prices")
    plt.grid(True)
    plt.legend(["stock"])
    plt.xlabel("Time")
    plt.ylabel("Prices")
    
    plt.subplot(2,1,2)
    plt.plot(RSI)
    plt.axhline(y=70, linewidth=1, color = 'r', linestyle='--')
    plt.axhline(y=30, linewidth=1, color = 'b', linestyle='--')
    plt.title("RSI Indicator")
    plt.grid(True)
    plt.legend(["RSI"])
    plt.xlabel("Time")
    plt.ylabel("Indicator")
    plt.savefig('Figure3.png', dpi=600, bbox_inches='tight')
    return None 

### Exponential Moving Average

In [ ]:
def price_ema_indicator(stock, window=20): 
    sgnl_alpa = window
    stock_ema  = stock.ewm(span=sgnl_alpa, min_periods=0, adjust=False, ignore_na=False).mean()
    
    plt.figure(4)
    plt.plot(stock,'k-')
    plt.plot(stock_ema,'r-')
    plt.title("Stock vs EWA")
    plt.grid(True)
    plt.legend(["stock", "EMA[{}]".format(window)])
    plt.xlabel("Time")
    plt.ylabel("Prices vs Indicator")
    plt.savefig('Figure4.png', dpi=600, bbox_inches='tight')
    return None